# PDAC ILP (Exact) Algorithm analysis
This notebook is used to analyze the performance of the ILP (exact) job scheduling algorithm. Specifically, it uses an relaxed ILP (inexact) scheduling algorithm, greedy heuristic scheduling algorithm and a naive scheduling algorithm for comparison. The peak demand above the curve (PDAC) is measured across five different trials for each specified job batch size and then plotted on a graph. 

In [ ]:
import sys
import os
import random
import json
import matplotlib.pyplot as plt
import numpy as np
import time

# Add the parent directory of AAC/ to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

Import the ILP (exact), relaxed ILP (inexact), greedy and naive algorithms from their respective files in the repository

In [ ]:
from PDAC.pdac_scheduling_ilp import solve_pdac_ilp
from PDAC.pdac_scheduling_lp import solve_pdac_lp
from PDAC.pdac_scheduling_greedy import solve_pdac_greedy
from PDAC.pdac_scheduling_naive import solve_pdac_naive

# Program Parameters
In the code below, you can specify the start time, end time and maximum length of each job in each batch of jobs. \
\
&emsp;&emsp; - 0 denotes 12am and 1400 denotes 12am of the following day. \
&emsp;&emsp; - Therefore, one day contains exactly 1440 minutes of job data
\
\
Additionally, you can specify the sizes of each job batch that is analyzed in this program. \
\
&emsp;&emsp; - The start size is the smallest batch size to be analyzed \
&emsp;&emsp; - The end size is the largest batch size (exclusive) to be analyzed \
&emsp;&emsp; - The step size is how much the job batch increases per analytical trial
\
\
For the exact algorithm, because it is an ILP, runtime increases exponentially as the job batch sizes increase. It is therefore advisable to keep the job batch sizes at or below 100 for this analysis

In [ ]:
start_time = 0
end_time = 1400
max_length = 700

start_size = 10
end_size = 100
step_size = 10

analysis_num = 0

# Resource Curve
The resource curve is represented by a list of values across the specified time period. Where at a given time t there will be exactly resources[t] amount of power available to the jobs. The data for these resources is available in the Input_Data folder. 
\
\
Additionally, each height is multiplied by a scaling factor. This is so that the resource curve better fits the job curve. Otherwise, the resource curve would be too large for any sort of significant analysis. This scaling factor remains constant for each of the trials and for every job batch size for the sake of consistency. 

In [ ]:
# Instantiate the resource curve
path = "../../Input_Data/resource_data.json"
with open(path, 'r') as file:
    data = json.load(file)

wind_energy = data['series'][1]['data']
solar_energy = data['series'][2]['data']
hydro_energy = data['series'][3]['data']

# Have 165 hours and you want minute by minute resolution. Therefore the total length of this list needs to be 165 * 60 in length
# The first 60 values of the list need to equal 0, the next need to equal 1. Therefore, i // 60
wind_energy_7_days = [0 for _ in range(165 * 60)]
for i in range(len(wind_energy_7_days)):
    wind_energy_7_days[i] = wind_energy[ i // 60 ]['value']

solar_energy_7_days = [0 for _ in range(165 * 60)]
for i in range(len(solar_energy_7_days)):
    solar_energy_7_days[i] = solar_energy[ i // 60 ]['value']

hydro_energy_7_days = [0 for _ in range(165 * 60)]
for i in range(len(hydro_energy_7_days)):
    hydro_energy_7_days[i] = hydro_energy[ i // 60 ]['value']


total = []
for i in range(165 * 60):
    total_sum = wind_energy_7_days[i] + solar_energy_7_days[i] + hydro_energy_7_days[i]
    total.append(total_sum)

day = 3
resources = total[(24 * day) + start_time : (24 * day) + end_time]

# # Implement a resource curve scaling factor to better fit the jobs
scale_factor = 0.05
resources = [r * scale_factor for r in resources]

# Algorithm Analysis
For each batch size, there will be five trials run on each algorithm. Each time, a random assortment of jobs will be provided to the algorithms. However, these jobs will be the same for each algorithm, so they will be tested on the same jobs. This is for the most accurate possible comparisons. 
\
\
Additionally, the time it takes for each algorihtm to run is measured. 

In [ ]:
# This is The list of job objects that will be scheduled
# They each have a release, deadline, duration and height
path = "../../Input_Data/job_data.json"
with open(path, 'r') as file:
    data = json.load(file)

# Randomly shuffle the jobs so that there is variation between trials
jobs_array = data['jobs']

pdac_inexact = []
pdac_exact = []
pdac_greedy = []
pdac_naive = []

final_data = []

for batch_size in range(start_size, end_size, step_size):
    exact_total = 0
    inexact_total = 0
    greedy_total = 0
    naive_total = 0
    
    print(f"\nBatch Size: {batch_size}")

    for trial in range(5):
        print(f"Trial #: {trial}")
        random.shuffle(jobs_array)
        
        start = time.time()
        exact = solve_pdac_ilp(jobs_array, resources, start_time, end_time, max_length, batch_size)[0]
        exact_total += exact
        end = time.time()
        exact_time = end - start
        print(f"Exact Objective: {exact}, Elapsed Time: {exact_time}")

        start = time.time()
        inexact = solve_pdac_lp(jobs_array, resources, start_time, end_time, max_length, batch_size)[0]
        inexact_total += inexact
        end = time.time()
        inexact_time = end - start
        print(f"Inexact Objective: {inexact}, Elapsed Time: {inexact_time}")

        start = time.time()
        greedy = solve_pdac_greedy(jobs_array, resources, start_time, end_time, max_length, batch_size)[0]
        greedy_total += greedy
        end = time.time()
        greedy_time = end - start
        print(f"Greedy Objective: {greedy}, Elapsed Time: {greedy_time}")

        start = time.time()
        naive = solve_pdac_naive(jobs_array, resources, start_time, end_time, max_length, batch_size)[0]
        naive_total += naive
        end = time.time()
        naive_time = end - start
        print(f"Naive Objective: {naive}, Elapsed Time: {naive_time}")

        trial_data = {
            "batch size": batch_size, 
            "trial #": trial,
            "exact objective val": exact, 
            "exact time": exact_time, 
            "naive obective val": naive, 
            "naive time": naive_time, 
            "inexact objective val": inexact, 
            "inexact time": inexact_time, 
            "greedy objective val": greedy, 
            "greedy time": greedy_time
        }
        final_data.append(trial_data)

    pdac_inexact.append(inexact_total / 5)
    pdac_exact.append(exact_total / 5)
    pdac_greedy.append(greedy_total / 5)
    pdac_naive.append(naive_total / 5)


In [ ]:
inexact_list = [x / 1000 for x in pdac_inexact]

greedy_list = [x / 1000 for x in pdac_greedy]

naive_list = [x / 1000 for x in pdac_naive]

exact_list = [x / 1000 for x in pdac_exact]

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))

# Plot data
job_graph_xvalues = [i for i in range(start_size, end_size, step_size)]
ax.plot(job_graph_xvalues, inexact_list, label="PDAC Inexact", linestyle='-', color = 'black', linewidth=1.5)
ax.plot(job_graph_xvalues, greedy_list, label="PDAC Greedy", linestyle='--', color = 'black', linewidth=1.5)
ax.plot(job_graph_xvalues, naive_list, label="PDAC Naive", linestyle=':', color = 'black', linewidth=1.5)
ax.plot(job_graph_xvalues, exact_list, label="PDAC Exact", linestyle='-.', color = 'black', linewidth=1.5)

ax.grid(True, linestyle='--', linewidth=0.25, color='black')
plt.tick_params(axis='both', labelsize=14)
ax.set_xticks(np.arange(start_size, end_size, step_size))  
ax.set_yticks(np.arange(0, max(naive_list)+2, 2))

ax.set_title("Algorithm Comparison by Batch Size", fontsize=36, fontname="Georgia", color='black')
ax.set_xlabel("Job Batch Size", fontsize=24, fontname='Verdana', color='black')
ax.set_ylabel("Peak Demand Above \n Curve (kWh)", fontsize=24, fontname='Verdana', color='black')
ax.legend(loc='upper left', prop={'family': 'Verdana', 'size': 14})

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))

# Set figure (outside) background
fig.patch.set_facecolor("#261c15")

# Set axes (plot area) background
ax.set_facecolor("#c2c2c2")

# Plot data
job_graph_xvalues = [i for i in range(start_size, end_size, step_size)]
ax.plot(job_graph_xvalues, inexact_list, label="PDAC Inexact", linestyle='-', color = '#f77f00', linewidth=2.5)
ax.plot(job_graph_xvalues, greedy_list, label="PDAC Greedy", linestyle='--', color = '#0081a7', linewidth=2.5)
ax.plot(job_graph_xvalues, naive_list, label="PDAC Naive", linestyle=':', color = '#9448bc', linewidth=2.5)
ax.plot(job_graph_xvalues, exact_list, label="PDAC Exact", linestyle='-.', color = '#564e58', linewidth=2.5)

ax.grid(True, linestyle='--', linewidth=0.25, color='black')
plt.tick_params(axis='both', labelsize=14)
ax.set_xticks(np.arange(start_size, end_size, step_size))
ax.set_yticks(np.arange(0, max(naive_list)+2, 2))

ax.set_title("Algorithm Comparison by Batch Size", fontsize=36, fontname="Georgia", color='white')
ax.set_xlabel("Job Batch Size", fontsize=24, fontname='Verdana', color='white')
ax.set_ylabel("Peak Demand Above \n Curve (kWh)", fontsize=24, fontname='Verdana', color='white')
ax.legend(loc='upper left', prop={'family': 'Verdana', 'size': 14})

ax.tick_params(colors='white')
for spine in ax.spines.values():
    spine.set_edgecolor('white')

plt.show()

In [ ]:
print(final_data)
job_graph_xvalues = np.array([i for i in range(start_size, end_size, step_size)])

plt.plot(job_graph_xvalues, exact_list, label="PDAC Exact", color='blue')
plt.plot(job_graph_xvalues, inexact_list, label="PDAC Inexact", color='orange')
plt.plot(job_graph_xvalues, greedy_list, label="PDAC Greedy", color='green')
plt.plot(job_graph_xvalues, naive_list, label="PDAC Naive", color='red')


plt.xlabel("Job Batch Size")
plt.ylabel("Power Demand Above Curve")
plt.legend()
plt.show()

In [ ]:
import csv
folder_path = r"C:\Users\moaxs\Desktop\REU 2025\PDM Project\Code\Graphs\Comparison\Output_Data\PDAC_Results"
# Write to a data csv file
with open(f"{folder_path}/exact_pdac_analysis_{analysis_num}.csv", "a", newline="") as csvfile:
    fieldnames = ['batch size', 'trial #', 'exact objective val', 'exact time', 'naive obective val', 'naive time', 'inexact objective val', 'inexact time', 'greedy objective val', 'greedy time']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Only write the header on the very first trial run
    writer.writeheader()
    writer.writerows(final_data)

    csvfile.close()